In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
%cd /content/drive/MyDrive

/content/drive/MyDrive


In [3]:
!git clone https://github.com/ultralytics/yolov5  # Clone the repo
%cd yolov5
!pip install -r requirements.txt  # Install dependencies

Cloning into 'yolov5'...
remote: Enumerating objects: 16965, done.
remote: Counting objects: 100% (160/160), done.
remote: Compressing objects: 100% (110/110), done.
remote: Total 16965 (delta 81), reused 101 (delta 50), pack-reused 16805 (from 1)
Receiving objects: 100% (16965/16965), 15.71 MiB | 10.30 MiB/s, done.
Resolving deltas: 100% (11619/11619), done.
Updating files: 100% (147/147), done.
/content/drive/MyDrive/yolov5
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 881.7/881.7 kB 21.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 5.0 MB/s eta 0:00:00


In [4]:
import torch

# Load the YOLOv5 model from your saved weights
model = torch.hub.load('ultralytics/yolov5', 'custom', path='/content/drive/MyDrive/yolov5_trained_model/weights/best.pt')  # Adjust the path if needed


/usr/local/lib/python3.10/dist-packages/torch/hub.py:295: UserWarning: You are about to download and run code from an untrusted repository. In a future release, this won't be allowed. To add the repository to your trusted list, change the command to {calling_fn}(..., trust_repo=False) and a command prompt will appear asking for an explicit confirmation of trust, or load(..., trust_repo=True), which will assume that the prompt is to be answered with 'yes'. You can also use load(..., trust_repo='check') which will only prompt for confirmation if the repo is not already trusted. This will eventually be the default behaviour
  warnings.warn(
Downloading: "https://github.com/ultralytics/yolov5/zipball/master" to /root/.cache/torch/hub/master.zip


Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


YOLOv5 🚀 2024-10-3 Python-3.10.12 torch-2.4.1+cu121 CPU

Fusing layers... 
Model summary: 157 layers, 7012822 parameters, 0 gradients, 15.8 GFLOPs
Adding AutoShape... 


In [5]:
# Define the path to your test image
image_path = '/content/drive/MyDrive/car_plate_yolo/val/Cars401.png'  # Adjust the path as necessary

# Perform inference
results = model(image_path)

# Print predictions
results.print()  # This will display the bounding boxes and confidence scores


/root/.cache/torch/hub/ultralytics_yolov5_master/models/common.py:892: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):
image 1/1: 300x400 1 car_plate
Speed: 704.1ms pre-process, 321.0ms inference, 33.4ms NMS per image at shape (1, 3, 480, 640)


In [6]:
# Get predictions
predictions = results.xyxy[0]  # Get predictions for the first image

# Extract bounding box coordinates
for *box, conf, cls in predictions:
    x1, y1, x2, y2 = map(int, box)  # Get the coordinates and convert them to integers
    print(f'Bounding Box: ({x1}, {y1}), ({x2}, {y2}) with confidence {conf:.2f}')


Bounding Box: (159, 220), (236, 241) with confidence 0.83


In [7]:
from PIL import Image
import cv2

# Load the image using OpenCV
image = cv2.imread(image_path)

# Loop through each detected bounding box
for *box, conf, cls in predictions:
    x1, y1, x2, y2 = map(int, box)

    # Crop the region of interest (ROI) for OCR
    roi = image[y1:y2, x1:x2]

    # Save or process the ROI as needed
    cv2.imwrite('/content/drive/MyDrive/car_plate_yolo/roi.jpg', roi)  # Save the cropped ROI


In [8]:
!apt-get install tesseract-ocr
!pip install pytesseract


Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  tesseract-ocr-eng tesseract-ocr-osd
The following NEW packages will be installed:
  tesseract-ocr tesseract-ocr-eng tesseract-ocr-osd
0 upgraded, 3 newly installed, 0 to remove and 49 not upgraded.
Need to get 4,816 kB of archives.
After this operation, 15.6 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/universe amd64 tesseract-ocr-eng all 1:4.00~git30-7274cfa-1.1 [1,591 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy/universe amd64 tesseract-ocr-osd all 1:4.00~git30-7274cfa-1.1 [2,990 kB]
Get:3 http://archive.ubuntu.com/ubuntu jammy/universe amd64 tesseract-ocr amd64 4.1.1-2.1build1 [236 kB]
Fetched 4,816 kB in 0s (10.9 MB/s)
Selecting previously unselected package tesseract-ocr-eng.
(Reading database ... 123620 files and directories currently installed.)
Preparing to unpack .../tesseract-ocr-e

In [9]:
import pytesseract

# Load the cropped ROI for OCR
roi_image_path = '/content/drive/MyDrive/car_plate_yolo/roi.jpg'  # Path to the cropped image
roi_image = Image.open(roi_image_path)

# Perform OCR
text = pytesseract.image_to_string(roi_image)

print("Detected Text:", text)


Detected Text: HR 26 BC Sia
HR 26 BC 5514



In [10]:
import torch
import cv2
from PIL import Image, ImageDraw, ImageFont
import matplotlib.pyplot as plt


In [11]:
# Load the original image using OpenCV
image = cv2.imread(image_path)

# Convert BGR to RGB for display
image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
# Initialize the font for displaying text
font = cv2.FONT_HERSHEY_SIMPLEX

# Loop through each detected bounding box
for *box, conf, cls in predictions:
    x1, y1, x2, y2 = map(int, box)  # Convert to integers

    # Crop the region of interest for OCR
    roi = image[y1:y2, x1:x2]

    # Perform OCR
    text = pytesseract.image_to_string(roi).strip()  # Get the detected text

    # Draw the bounding box on the original image
    cv2.rectangle(image_rgb, (x1, y1), (x2, y2), (255, 0, 0), 2)  # Blue box

    # Overlay the detected text on the image
    cv2.putText(image_rgb, text, (x1, y1 - 10), font, 0.5, (255, 0, 0), 2)  # Blue text

# Display the final image with bounding boxes and text
plt.figure(figsize=(10, 10))
plt.imshow(image_rgb)
plt.axis('off')  # Hide axes
plt.show()


In [12]:
# Save the final image
output_path = '/content/drive/MyDrive/car_plate_detection_result.jpg'  # Adjust the path as necessary
cv2.imwrite(output_path, cv2.cvtColor(image_rgb, cv2.COLOR_RGB2BGR))  # Convert back to BGR for saving


True